In [ ]:
import mediapipe as mp
import cv2

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

In [ ]:
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0) #Checks for camera
 
mpHands = mp.solutions.hands #detects hand/finger
hands = mpHands.Hands()   #complete the initialization configuration of hands
mpDraw = mp.solutions.drawing_utils
 
#To access speaker through the library pycaw 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0
 
volMin,volMax = volume.GetVolumeRange()[:2]
 
while True:
    success,img = cap.read() #If camera works capture an image
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb
    
    #Collection of gesture information
    results = hands.process(imgRGB) #completes the image processing.
 
    lmList = [] #empty list
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy]) #adding to the empty list 'lmList'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x1,y1 = lmList[4][1],lmList[4][2]  #thumb
        x2,y2 = lmList[8][1],lmList[8][2]  #index finger
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  #create a line b/w tips of index finger and thumb
 
        length = hypot(x2-x1,y2-y1) #distance b/w tips using hypotenuse
 # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
        vol = np.interp(length,[30,350],[volMin,volMax]) 
        volbar=np.interp(length,[30,350],[400,150])
        volper=np.interp(length,[30,350],[0,100])
        
        
        print(vol,int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(0,0,255),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(0,0,255),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(0, 255, 98),3)
        #tell the volume percentage ,location,font of text,length,rgb color,thickness
    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord(' '): #By using spacebar delay will stop
        break
        
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window

-65.25 27
-46.079523903380775 124
-57.000838115799915 70
-57.56768985501646 67
-56.33195306431085 73
-53.418803865319795 88
-29.212313396943635 206
-27.633693675979217 214
-25.50278354208249 224
-25.902037594198177 222
-25.36384063954558 225
-24.80462494105503 228
-25.01045699608401 227
-25.280315098639782 226
-24.908321908849345 227
-24.6552962810609 229
-25.02525835400246 227
-24.763567620593008 228
-25.953278320403314 222
-25.77008421210384 223
-26.774610854938288 218
-26.77088144315246 218
-26.946502266526522 217
-26.577379936534562 219
-27.081491365919888 217
-26.99941778066247 217
-24.554824231987908 229
-25.01045699608401 227
-23.624813087166324 234
-23.373356261870427 235
-21.939370367889943 242
-22.21562929033103 241
-22.116335245913 241
-22.02945887307137 241
-20.883607194190027 247
-21.081657212705473 246
-20.73845228623327 248
-20.20581313606563 250
-20.027335136813093 251
-20.48042284176286 249
-19.84301939723835 252
-19.001378755982316 256
-17.82946682195478 262
-16.74085

In [ ]:
import pyttsx3

In [ ]:
r = pyttsx3.init()

robot = input('Digite sua frase: ')

r.say(robot)

r.runAndWait()

In [ ]:
#le o texto que esta guardado em um arquivo .txt
f = open('feynman.txt', 'r', encoding="utf8")
texto = f.read()

speaker = pyttsx3.init() #inicia serviço biblioteca
voices = speaker.getProperty('voices') #metodo de voz

#ver as vozes instaladas na maquina
for voice in voices:
    print(voice, voice.id) #traz os idiomas de voz instalados em sua maquina
    
speaker.setProperty('voice', voices[0].id) #define a voz padrao, no meu caso o portugues era o[0] (iniciando do zero)
rate = speaker.getProperty('rate')
speaker.setProperty('rate', rate-25) #muda velocidade da leitura, quando menor mais lento

print(texto) #escreve o texto na tela
speaker.say(texto) #define o texto que será lido
speaker.runAndWait() #le o texto
f.close() #fecha o modo deleitura do arquivo txt

<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_PT-BR_MARIA_11.0
          name=Microsoft Maria Desktop - Portuguese(Brazil)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_PT-BR_MARIA_11.0
<Voice id=HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
          name=Microsoft Zira Desktop - English (United States)
          languages=[]
          gender=None
          age=None> HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
17 de Outubro, 1946

D’Arline,

Eu te adoro, meu docinho.

Eu sei o quanto você gosta de ouvir isso, mas não escrevo apenas porque você gosta: escrevo porque isso me aquece por dentro só de te escrever.

Passou um tempo terrivelmente longo desde a última vez que te escrevi — quase dois anos. Mas eu sei que você vai me perdoar porque você entende como eu sou, teimoso e realista. Pensei que não 